## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Blackwater,-23.5833,148.8833,71.98,81,100,12.30,AU,light rain
1,1,Channel-Port Aux Basques,47.5702,-59.1367,42.37,95,100,32.61,CA,light rain
2,2,Ushuaia,-54.8000,-68.3000,39.20,81,75,19.57,AR,light snow
3,3,Kununurra,-15.7667,128.7333,91.40,43,4,10.36,AU,clear sky
4,4,Rundu,-17.9333,19.7667,64.54,60,2,3.91,NaN,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
6,6,Cabo San Lucas,22.8909,-109.9124,86.00,21,1,17.27,MX,clear sky
12,12,Rikitea,-23.1203,-134.9692,77.99,65,0,18.79,PF,clear sky
13,13,Teluk Nibung,3.0010,99.8164,81.43,80,95,3.51,ID,overcast clouds
16,16,Kapaa,22.0752,-159.3190,77.00,57,90,17.27,US,overcast clouds
18,18,Puri,19.8000,85.8500,81.10,78,0,5.59,IN,clear sky
20,20,Carutapera,-1.1950,-46.0200,75.52,95,100,5.91,BR,overcast clouds
24,24,Lorengau,-2.0226,147.2712,81.43,78,100,4.56,PG,overcast clouds
30,30,Atuona,-9.8000,-139.0333,80.11,79,76,8.12,PF,light rain
32,32,Duku,11.2361,4.9073,81.03,12,36,4.92,NG,scattered clouds
33,33,Samut Sakhon,13.5833,100.3000,82.99,73,100,7.20,TH,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                221
City                   221
Lat                    221
Lng                    221
Max Temp               221
Humidity               221
Cloudiness             221
Wind Speed             221
Country                220
Current Description    221
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Cabo San Lucas,MX,86.00,clear sky,22.8909,-109.9124,
12,Rikitea,PF,77.99,clear sky,-23.1203,-134.9692,
13,Teluk Nibung,ID,81.43,overcast clouds,3.0010,99.8164,
16,Kapaa,US,77.00,overcast clouds,22.0752,-159.3190,
18,Puri,IN,81.10,clear sky,19.8000,85.8500,
20,Carutapera,BR,75.52,overcast clouds,-1.1950,-46.0200,
24,Lorengau,PG,81.43,overcast clouds,-2.0226,147.2712,
30,Atuona,PF,80.11,light rain,-9.8000,-139.0333,
32,Duku,NG,81.03,scattered clouds,11.2361,4.9073,
33,Samut Sakhon,TH,82.99,overcast clouds,13.5833,100.3000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
       
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Cabo San Lucas,MX,86.00,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
12,Rikitea,PF,77.99,clear sky,-23.1203,-134.9692,Pension Maro'i
13,Teluk Nibung,ID,81.43,overcast clouds,3.0010,99.8164,Hotel Assyifa
16,Kapaa,US,77.00,overcast clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
18,Puri,IN,81.10,clear sky,19.8000,85.8500,OYO 1577 Hotel Shree Hari Grand


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))